In [ ]:
import numpy as np
import pandas as pd

from utils import get_fields

In [ ]:
# 'mm' for multimodal (aesthetic emotions), 'mf' for music-focused (perceptual descriptors)
experiment = "mm"

ordered_fields, _ = get_fields(experiment)
ordered_fields = list(ordered_fields) # not alphabetical order

## Load control ratings

In [ ]:
control_ratings_df = pd.read_csv(f"{experiment}_control_ratings.csv")

## Inter-rater agreement != reliability

In [ ]:
r_star_wgs = {field: [] for field in ordered_fields}
for field in ordered_fields:

    # compute r_star_wg for each stimulus and average over all stimuli
    for stimulus_id in control_ratings_df["stimulus_id"].unique():
        df_field = control_ratings_df[["prolific_id", "stimulus_id", field]].copy()
        df_field = df_field[df_field["stimulus_id"] == stimulus_id]

        # r∗wg with the rectangular null and maximum dissensus null distributions
        # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5427087/#sec-a.h.ctitle
        # This index avoids the problem of non-linearity and corresponding inflation potential of rwg(j) 
        # and addresses the problem of inadmissible values.
        
        S_x = df_field[[field]].var().values[0] # observed variance
        X_U = 7 # maximum possible value
        X_L = 1 # minimum possible value
        sigma_mv = 0.5*(X_U**2 + X_L**2) - (0.5*(X_U + X_L))**2 # maximum dissensus variance (ratings uniformly dist. at the two extremes)
        # the above is equivalent to sigma_mv = np.std([X_U, X_L])**2
        r_star_wgs[field].append(1 - (S_x / sigma_mv))

    print(f"{field} - r*_wg: {np.mean(r_star_wgs[field]):.2f} ± {np.std(r_star_wgs[field]):.2f}")